In [1]:
%pip install git+https://github.com/bigscience-workshop/petals

  Cloning https://github.com/bigscience-workshop/petals to /tmp/pip-req-build-g49wjr7q
  Running command git clone --filter=blob:none --quiet https://github.com/bigscience-workshop/petals /tmp/pip-req-build-g49wjr7q
  Resolved https://github.com/bigscience-workshop/petals to commit d59c15c5787488005f12162a40930dd284551e02
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 79.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.7/242.7 kB 27.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import torch
from transformers import AutoTokenizer
from petals import AutoDistributedModelForCausalLM

model_name = "petals-team/StableBeluga2"
# You could also use "meta-llama/Llama-2-70b-chat-hf" or any other supported model from 🤗 Model Hub

tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False, add_bos_token=False)
model = AutoDistributedModelForCausalLM.from_pretrained(model_name)
model = model.cuda()

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Nov 22 06:24:42.775 [INFO] Make sure you follow the LLaMA's terms of use: https://bit.ly/llama2-license for LLaMA 2, https://bit.ly/llama-license for LLaMA 1
Nov 22 06:24:42.781 [INFO] Using DHT prefix: StableBeluga2-hf


In [3]:
fake_token = tokenizer("^")["input_ids"][0]  # Workaround to make tokenizer.decode() keep leading spaces

text = "What is a good chatbot? Answer:"
prefix = tokenizer(text, return_tensors="pt")["input_ids"].cuda()

with model.inference_session(max_length=30) as sess:
    for i in range(20):
        # Prefix is passed only for the 1st token of the outputs
        inputs = prefix if i == 0 else None

        # Let's use sampling with temperature = 0.9 and top_p = 0.6 to get more diverse results
        outputs = model.generate(inputs, max_new_tokens=1, session=sess,
                                 do_sample=True, temperature=0.9, top_p=0.6)

        text += tokenizer.decode([fake_token, outputs[0, -1].item()])[1:]
        print(text)

Nov 22 06:25:20.306 [INFO] Route found: 0:40 via …CMBDje => 40:80 via …xxYbSH
Nov 22 06:25:30.832 [INFO] NumExpr defaulting to 2 threads.


What is a good chatbot? Answer: A
What is a good chatbot? Answer: A good
What is a good chatbot? Answer: A good chat
What is a good chatbot? Answer: A good chatbot
What is a good chatbot? Answer: A good chatbot is
What is a good chatbot? Answer: A good chatbot is one
What is a good chatbot? Answer: A good chatbot is one that
What is a good chatbot? Answer: A good chatbot is one that can
What is a good chatbot? Answer: A good chatbot is one that can understand
What is a good chatbot? Answer: A good chatbot is one that can understand and
What is a good chatbot? Answer: A good chatbot is one that can understand and respond
What is a good chatbot? Answer: A good chatbot is one that can understand and respond to
What is a good chatbot? Answer: A good chatbot is one that can understand and respond to user
What is a good chatbot? Answer: A good chatbot is one that can understand and respond to user queries
What is a good chatbot? Answer: A good chatbot is one that can understand and respond t

In [4]:
with model.inference_session(max_length=512) as sess:
    while True:
        prompt = input('Human: ')
        if prompt == "":
            break
        prefix = f"Human: {prompt}\nFriendly AI:"
        prefix = tokenizer(prefix, return_tensors="pt")["input_ids"].cuda()
        print("Friendly AI:", end="", flush=True)

        while True:
            outputs = model.generate(prefix, max_new_tokens=1, session=sess,
                                     do_sample=True, temperature=0.9, top_p=0.6)
            outputs = tokenizer.decode([fake_token, outputs[0, -1].item()])[1:]

            # Now, let's print one new token at a time
            print(outputs, end="", flush=True)

            if "\n" in outputs or "</s>" in outputs:
                break
            prefix = None  # Prefix is passed only for the 1st token of the bot's response

Human: describe the sun
Friendly AI:

Nov 22 06:30:19.211 [INFO] Route found: 0:40 via …HfQWVM => 40:80 via …2mj7xE


 The sun is a bright, warm, and life-giving ball of light in the sky that provides energy and warmth to our planet. It is a star that emits light and heat, and its presence allows plants to grow, and ecosystems to thrive. The sun is essential for life on Earth, and its energy is harnessed in various ways, such as through solar power. The sun is at the center of our solar system, and its gravitational pull keeps planets, including Earth, in their orbits.
Human: 


In [5]:
model

DistributedLlamaForCausalLM(
  (model): DistributedLlamaModel(
    (embed_tokens): Embedding(32000, 8192, padding_idx=0)
    (layers): RemoteSequential(modules=StableBeluga2-hf.0..StableBeluga2-hf.79)
    (norm): LlamaRMSNorm()
  )
  (lm_head): LMHead()
)

In [6]:
first_five_layers = model.model.layers[0:5]
first_five_layers

RemoteSequential(modules=StableBeluga2-hf.0..StableBeluga2-hf.4)

In [7]:
dummy_inputs = torch.randn(1, 3, model.config.hidden_size, dtype=torch.bfloat16, requires_grad=True)
outputs = first_five_layers(dummy_inputs)
outputs

tensor([[[-0.9844, -0.6914,  0.8047,  ..., -0.2539,  0.8906,  1.5703],
         [ 0.2178,  0.4082,  0.7383,  ...,  0.1118,  0.0386, -0.6797],
         [ 0.3262,  1.4688,  1.2656,  ..., -0.8125, -0.6016, -0.0525]]],
       dtype=torch.bfloat16, grad_fn=<_RemoteSequentialAutogradFunctionBackward>)

In [8]:
loss = torch.mean((outputs - torch.ones_like(outputs)) ** 2)
loss.backward()  # backpropagate through the internet
print("Grad w.r.t. inputs:", dummy_inputs.grad.flatten())

Grad w.r.t. inputs: tensor([-1.8597e-04, -1.4400e-04,  1.7881e-05,  ..., -1.3638e-04,
        -1.3065e-04, -7.7724e-05], dtype=torch.bfloat16)


In [9]:
inputs = tokenizer("A quick brown fox", return_tensors="pt")["input_ids"].cuda()
outputs = model.generate(inputs, max_new_tokens=7)
print("generated:", tokenizer.decode(outputs[0]))

Nov 22 06:38:16.232 [INFO] Route found: 0:40 via …CMBDje => 40:80 via …xxYbSH


generated: A quick brown fox jumps over the lazy dog.


In [10]:
model = AutoDistributedModelForCausalLM.from_pretrained(model_name, tuning_mode='deep_ptune', pre_seq_len=3)
model = model.cuda()

Nov 22 06:38:23.997 [INFO] Make sure you follow the LLaMA's terms of use: https://bit.ly/llama2-license for LLaMA 2, https://bit.ly/llama-license for LLaMA 1
Nov 22 06:38:23.999 [INFO] Using DHT prefix: StableBeluga2-hf


In [11]:
opt = torch.optim.Adam(model.parameters(), lr=1e-3)

the_fox_is_innocent = tokenizer("A quick brown fox did not jump over the lazy dog", return_tensors="pt")["input_ids"].cuda()
for i in range(12):
    loss = model(input_ids=the_fox_is_innocent, labels=the_fox_is_innocent).loss
    print(f"loss[{i}] = {loss.item():.3f}")

    opt.zero_grad()
    loss.backward()
    opt.step()
    print("opt.step()")

loss[0] = 2.478
opt.step()


Nov 22 06:39:00.293 [WARN] [petals.client.sequential_autograd.sequential_forward:102] Caught exception when running forward via RemoteSpanInfo(peer_id=<libp2p.peer.id.ID (12D3KooWDmm5oA1mmPPVGKHsXWvYGxbEkTRCR1jFptYentpqDcUi)>, start=0, end=26, server_info=ServerInfo(state=<ServerState.ONLINE: 2>, throughput=141.78342222446955, start_block=None, end_block=None, public_name='donutmaster', version='2.2.0', network_rps=708.9171111223477, forward_rps=3402.463605342033, inference_rps=329.8703151801364, adapters=(), torch_dtype='bfloat16', quant_type='nf4', using_relay=True, cache_tokens_left=1703936, next_pings={'QmNSatwUFbjgAQ5C7ZFDpzcCuHaFHhvp6Zi4s44zaDb6mP': 0.2290173780694975, 'QmW6BwtMgiy9zc1zhoUJYD352zefa1VTAkPFYGCQHfQWVM': 0.2326748813072357, '12D3KooWRch61G1xsGcwkBqp7zroShGGdFSTtzbht6zQyJ7kNBGr': 0.18762797418431984, '12D3KooWRShgq87dhpsCJH1KNTwWK4Vnh7du21hhtegg7GMop8rK': 0.18312056342987412, 'QmQwW6f9chfeYQgte7nTC4WnGnVXp4UpzDW399b7CMBDje': 0.2276427098679305, '12D3KooWNgGTDTxuPcNmm

loss[1] = 4.394
opt.step()
loss[2] = 2.034
opt.step()
loss[3] = 2.167
opt.step()
loss[4] = 2.776
opt.step()
loss[5] = 1.562
opt.step()
loss[6] = 1.983
opt.step()
loss[7] = 1.136
opt.step()
loss[8] = 1.195
opt.step()
loss[9] = 0.640
opt.step()
loss[10] = 0.212
opt.step()
loss[11] = 0.054
opt.step()


In [12]:
inputs = tokenizer("A quick brown fox", return_tensors="pt")["input_ids"].cuda()
outputs = model.generate(inputs, max_new_tokens=7)
print("generated:", tokenizer.decode(outputs[0]))

Nov 22 06:41:09.187 [INFO] Route found: 0:40 via …aDb6mP => 40:80 via …2mj7xE


generated: A quick brown fox did not jump over the lazy dog


In [13]:
import torch.nn as nn
import torch.nn.functional as F

model = AutoDistributedModelForCausalLM.from_pretrained(model_name)
model = model.cuda()

class LLMBasedClassifier(nn.Module):
  def __init__(self, model):
    super().__init__()
    self.distributed_layers = model.transformer.h
    self.adapter = nn.Sequential(nn.Linear(model.config.hidden_size, 32), nn.Linear(32, model.config.hidden_size))
    self.head = nn.Linear(model.config.hidden_size, 2)

  def forward(self, embeddings):
    mid_block = len(self.distributed_layers) // 2
    hidden_states = self.distributed_layers[:mid_block](embeddings)
    hidden_states = self.adapter(hidden_states)
    hidden_states = self.distributed_layers[mid_block:](hidden_states)
    pooled_states = torch.mean(hidden_states, dim=1)
    return self.head(pooled_states)

Nov 22 06:41:18.534 [INFO] Make sure you follow the LLaMA's terms of use: https://bit.ly/llama2-license for LLaMA 2, https://bit.ly/llama-license for LLaMA 1
Nov 22 06:41:18.536 [INFO] Using DHT prefix: StableBeluga2-hf


In [14]:
classifier = LLMBasedClassifier(model).cuda()
opt = torch.optim.Adam(classifier.parameters(), 3e-5)
inputs = torch.randn(3, 2, model.config.hidden_size, device='cuda')
labels = torch.tensor([1, 0, 1], device='cuda')

for i in range(5):
  loss = F.cross_entropy(classifier(inputs), labels)
  print(f"loss[{i}] = {loss.item():.3f}")
  opt.zero_grad()
  loss.backward()
  opt.step()

print('predicted:', classifier(inputs).argmax(-1))  # l, o, l

loss[0] = 1.795
loss[1] = 0.066
loss[2] = 0.019
loss[3] = 0.004
loss[4] = 0.003
predicted: tensor([1, 0, 1], device='cuda:0')


In [15]:
from hivemind import get_logger

logger = get_logger()

fake_token = tokenizer("^")["input_ids"][0]  # Workaround to make tokenizer.decode() keep leading spaces

text = "What is a good chatbot? Answer:"
token_ids = tokenizer(text, return_tensors="pt")["input_ids"].cuda()
max_length = 100
with torch.inference_mode():
    with model.inference_session(max_length=max_length) as sess:
        while len(text) < max_length:
            embs = model.transformer.word_embeddings(token_ids)
            embs = model.transformer.word_embeddings_layernorm(embs)

            h = sess.step(embs)
            h_last = model.transformer.ln_f(h[:, -1])
            logits = model.lm_head(h_last)

            next_token = logits.argmax(dim=-1)
            text += tokenizer.decode([fake_token, next_token.item()])[1:]
            token_ids = next_token.reshape(1, 1)
            logger.info(text)

Nov 22 06:42:57.684 [INFO] Route found: 0:40 via …CMBDje => 40:80 via …2mj7xE
Nov 22 06:43:00.332 [INFO] What is a good chatbot? Answer: A
Nov 22 06:43:00.756 [INFO] What is a good chatbot? Answer: A good
Nov 22 06:43:01.132 [INFO] What is a good chatbot? Answer: A good chat
Nov 22 06:43:01.515 [INFO] What is a good chatbot? Answer: A good chatbot
Nov 22 06:43:01.894 [INFO] What is a good chatbot? Answer: A good chatbot is
Nov 22 06:43:02.274 [INFO] What is a good chatbot? Answer: A good chatbot is one
Nov 22 06:43:02.651 [INFO] What is a good chatbot? Answer: A good chatbot is one that
Nov 22 06:43:03.042 [INFO] What is a good chatbot? Answer: A good chatbot is one that can
Nov 22 06:43:03.419 [INFO] What is a good chatbot? Answer: A good chatbot is one that can understand
Nov 22 06:43:03.797 [INFO] What is a good chatbot? Answer: A good chatbot is one that can understand user
Nov 22 06:43:04.177 [INFO] What is a good chatbot? Answer: A good chatbot is one that can understand user inp